### Тестирование моделей, поиск решения

### Пункт по поиску решений содержит следующий план рассмотрения моделей:
- Расстояние Левенштейна
- TF-IDF и косинусное подобие
- Манхэттенское расстояние
- Евклидово расстояние
- Перевод и векторизация
- Мультиязычная модель
- Поиск сходства по Faiss

In [1]:
pip install Levenshtein

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [3]:
import numpy as np
import pandas as pd
import torch
import faiss
from sentence_transformers import SentenceTransformer, util
from translate import Translator
import logging
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances

In [4]:
logging.getLogger().setLevel(logging.DEBUG)

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [5]:
# Создадим небольшой тестовый набор на русском и английском языках

In [6]:
query = 'Tambov'

name_ru = ['Томбов', 'Тамбов', 'Tомбофф']
name_en = ['tambopeu', 'Tombovf', 'Tambo']
corpus = name_ru + name_en

### Расстояние Левенштейна

In [7]:
def calculate_levenshtein_similarity(query, words):
    similarities = []
    for word in words:
        comparison = fuzz.ratio(word, query)
        similarities.append(round(comparison, 2))
    return similarities

levenshtein_similarity_ru = calculate_levenshtein_similarity(query, name_ru)
print(f'Сходство Левенштейна с русским текстом: {levenshtein_similarity_ru}')

levenshtein_similarity_en = calculate_levenshtein_similarity(query, name_en)
print(f'Сходство Левенштейна с английским текстом: {levenshtein_similarity_en}')

Сходство Левенштейна с русским текстом: [0, 0, 15]
Сходство Левенштейна с английским текстом: [57, 77, 91]


По результатам найденного нормальзованного неразрывного сходства расстояния Левенштейна, необходимо сделать предварительный перевод текста. Данная модель справилась лучше с английским текстом Tambo, которое явлется коротким и самым подходящим запросу. Следовательно можно сделать вывод, что модель справляется больше с сходством по длине слова.

### TF-IDF и косинусное подобие

In [8]:
# создадим функцию для векторизации с n-граммами
def calculate_similarity(query, names, language='en', ngram_range=(2, 3)):
    vectorizer = TfidfVectorizer(analyzer="char", ngram_range=ngram_range)
    tfidf_matrix = vectorizer.fit_transform(names)
    vector_query = vectorizer.transform([query]).toarray()[0]

    # Циклом вычислим косинусное сходство между запросом и списком
    similarities = []
    for name in names:
        vector_name = vectorizer.transform([name]).toarray()[0]
        cosine_similarity = 1 - distance.cosine(vector_name, vector_query)
        similarities.append(round(cosine_similarity, 2))

    return similarities


similarity_ru = calculate_similarity('Tambov', name_ru, language='ru')
print(f'Сходство с русскими текстом: {similarity_ru}')


similarity_en = calculate_similarity('Tambov', name_en, language='en')
print(f'Сходство с английским текстом: {similarity_en}')

Сходство с русскими текстом: [1, 1, 1]
Сходство с английским текстом: [0.47, 0.44, 0.79]


C:\Users\User\AppData\Roaming\Python\Python39\site-packages\scipy\spatial\distance.py:622: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


По результату мы видим, что необходимо переводить текст, так как с русским тектстом вектор не справился, по английскому тексту может фиксировать семантическое сходство.

### Манхэттенское расстояние

In [9]:
# напишем функцию для векторизации с n-граммами и метрикой манхэтэн
def calculate_manhattan_similarity(query, names, metric='cosine', ngram_range=(2, 3)):
    vectorizer = TfidfVectorizer(analyzer="char", ngram_range=ngram_range)
    tfidf_matrix = vectorizer.fit_transform(names)
    vector_query = vectorizer.transform([query]).toarray()[0]
    # добавим единицу чтобы при делении на ноль не получился ноль
    distances_result = 1 / (1 + manhattan_distances([vector_query], tfidf_matrix)[0])
    similarities = [round(similarity, 2) for similarity in distances_result]

    return similarities
    
manhattan_similarity_ru = calculate_manhattan_similarity('Tambov', name_ru, metric='manhattan')
print(f'Манхэттенское расстояние русского текста: {manhattan_similarity_ru}')

manhattan_similarity_en = calculate_manhattan_similarity('Tambov', name_en, metric='manhattan')
print(f'Манхэттенское расстояние английского текста: {manhattan_similarity_en}') 

Манхэттенское расстояние русского текста: [0.25, 0.25, 0.24]
Манхэттенское расстояние английского текста: [0.23, 0.21, 0.41]


Манхэттенское расстояние более устойчиво для текста с опечатками, но судя по русскому тексту возможно оно плохо улавливает семантическое сходство.

### Евклидово расстояние

In [10]:
# напишем функцию для векторизации с n-граммами и метрикой евклидово
def calculate_euclidean_similarity(query, names, metric='cosine', ngram_range=(2, 3)):
    vectorizer = TfidfVectorizer(analyzer="char", ngram_range=ngram_range)
    tfidf_matrix = vectorizer.fit_transform(names)
    vector_query = vectorizer.transform([query]).toarray()[0]
    distances_result = 1 / (1 + euclidean_distances([vector_query], tfidf_matrix)[0])
    similarities = [round(similarity, 2) for similarity in distances_result]

    return similarities
    
euclidean_similarity_ru = calculate_euclidean_similarity('Tambov', name_ru, metric='euclidean')
print(f'Евклидово расстояние русского текста: {euclidean_similarity_ru}')

euclidean_similarity_en = calculate_euclidean_similarity('Tambov', name_en, metric='euclidean')
print(f'Евклидово расстояние английского текста: {euclidean_similarity_en}')

Евклидово расстояние русского текста: [0.5, 0.5, 0.5]
Евклидово расстояние английского текста: [0.49, 0.49, 0.61]


Евклидово расстояние показало практически одинаковые метрики английского и русского текста, так же мы видим что оно учитывает длину слова близкое к запросу. Работает хуже чем Манхэтенское расстояние.

Из всех вышеперчисленных моделей, лучший результат показало Манхэттенское расстояние.

### Перевод и векторизация

In [11]:
translator = Translator(from_lang='ru', to_lang = 'en')
tranlator_query = translator.translate('Тоамбовв').split()
tranlator_query

['Toambov']

In [12]:
manhattan_similarity_query = calculate_manhattan_similarity('Tambov', tranlator_query, metric='manhattan')
print(f'Манхэттенское расстояние текста c переводом: {manhattan_similarity_query}') 

Манхэттенское расстояние текста c переводом: [0.36]


С помощью трансляции Манхэтенское рассстояние справляется, но хотелось бы лучше увидеть опечатки текста. Поэтому рассмотрим дальше мультиязычные модели.

### Мультиязычная модель

In [13]:
# напишем функции для двух версий многоязычной модели семантического сходства, версии которых поддерживают 50 языков
def similarity_model(query, names, model, model_name):
    embeddings = model.encode(names)
    query_embedding = model.encode(query)

    results = util.semantic_search(query_embedding, embeddings)[0]

    print(f'Многоязычная модель версии: {model_name}', sep='\n')
    for result in results:
        idx = int(result['corpus_id'])
        score = result['score']
        print(f'City: {names[idx]}, Score: {round(score, 2)}')

model_par = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
similarity_model(query, corpus, model_par, 'paraphrase-multilingual-mpnet-base-v2')

model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
similarity_model(query, corpus, model, 'distiluse-base-multilingual-cased-v2')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Многоязычная модель версии: paraphrase-multilingual-mpnet-base-v2
City: Tambo, Score: 0.95
City: Тамбов, Score: 0.92
City: tambopeu, Score: 0.89
City: Томбов, Score: 0.6
City: Tомбофф, Score: 0.57
City: Tombovf, Score: 0.44


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Многоязычная модель версии: distiluse-base-multilingual-cased-v2
City: Тамбов, Score: 0.91
City: Томбов, Score: 0.89
City: Tambo, Score: 0.87
City: Tомбофф, Score: 0.79
City: Tombovf, Score: 0.78
City: tambopeu, Score: 0.77


Многоязычная модель версии  универсального кодировщика предложений с расширенными знаниями показал результат лучше, чем многоязычная версия обученная на параллельных данных. Скор города Тамбов на первом месте второй модели.

### Поиск сходства по Faiss

In [14]:
# напишем функцию для использования faiss с евклидовой метрикой расстояния L2
def faiss_similarity_search(query_embedding, embeddings):
    # Преобразуем  данные во float32 для faiss
    embeddings = np.array(embeddings, dtype=np.float32)
    query_embedding = np.array(query_embedding, dtype=np.float32)

    # Создадим экземпляр индекса
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    # Выполним поиск по сходству
    k = len(embeddings)
    distances, indices = index.search(np.expand_dims(query_embedding, axis=0), k)

    # Преобразуем все в список
    indices = indices.flatten().tolist()
    distances = distances.flatten().tolist()

    return indices, distances

# Используем многоязычную модель
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
embeddings = model.encode(corpus)
query_embedding = model.encode(query)
# выполним сходство по faiss
indices, distances = faiss_similarity_search(query_embedding, embeddings) 
print('Результат по FAISS:', sep='\n')
for idx, dist in zip(indices, distances):
    print(f'City: {corpus[idx]}, Distance: {round(dist, 2)}')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Результат по FAISS:
City: Tambo, Distance: 0.83
City: Тамбов, Distance: 1.37
City: tambopeu, Distance: 1.46
City: Томбов, Distance: 4.47
City: Tомбофф, Distance: 4.76
City: Tombovf, Distance: 5.98


При испоьзовании Faiss мы видим что название города без опечаток стоит на втором месте. Результаты не плохие, но лучше работает мультиязычная модель версии distiluse-base-multilingual-cased-v2.